In [50]:
import open3d as o3d
import numpy as np

In [51]:
def load_point_clouds(voxel_size=0.0):
    pcds = []
    for i in range(2,4):
        pcd = o3d.io.read_point_cloud(f"/home/ppak/GitHub/CV-Robotics/outputs/point_c/{i}.pcd")
        pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
        pcd_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
        pcds.append(pcd_down)
    return pcds

In [52]:
def pairwise_registration(source, target, max_correspondence_distance_coarse,
                          max_correspondence_distance_fine):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp

In [53]:
def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id],
                max_correspondence_distance_coarse,
                max_correspondence_distance_fine)
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

In [54]:

voxel_size=0.02
pcds_down = load_point_clouds(voxel_size)
o3d.visualization.draw_geometries(pcds_down)

In [55]:


print("Full registration ...")
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)



Full registration ...
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.5981, RMSE 0.1080
[Open3D DEBUG] Residual : 1.04e-02 (# of elements : 4548)
[Open3D DEBUG] ICP Iteration #1: Fitness 0.6761, RMSE 0.1008
[Open3D DEBUG] Residual : 8.40e-03 (# of elements : 5141)
[Open3D DEBUG] ICP Iteration #2: Fitness 0.7687, RMSE 0.1128
[Open3D DEBUG] Residual : 8.01e-03 (# of elements : 5845)
[Open3D DEBUG] ICP Iteration #3: Fitness 0.7656, RMSE 0.1038
[Open3D DEBUG] Residual : 3.14e-03 (# of elements : 5822)
[Open3D DEBUG] ICP Iteration #4: Fitness 0.7550, RMSE 0.0985
[Open3D DEBUG] Residual : 2.11e-03 (# of elements : 5741)
[Open3D DEBUG] ICP Iteration #5: Fitness 0.7568, RMSE 0.0933
[Open3D DEBUG] Residual : 1.91e-03 (# of elements : 5755)
[Open3D DEBUG] ICP Iteration #6: Fitness 0.7596, RMSE 0.0892
[Open3D DEBUG] Residual : 1.85e-03 (# of elements : 5776)
[Open3D DEBUG] ICP Iteration #7: Fitness 0.7600, RMSE 0.0860
[Open3D DEBUG] Residual : 1.81e-03 (# of elements : 5779)
[

In [56]:
print("Optimizing PoseGraph ...")
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)

Optimizing PoseGraph ...
[Open3D DEBUG] Validating PoseGraph - finished.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 3.334500
[Open3D DEBUG] [Initial     ] residual : 1.225810e-28, lambda : 3.705000e-02
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 3.334500
[Open3D DEBUG] [Initial     ] residual : 1.225810e-28, lambda : 3.705000e-02
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] CompensateReferencePoseGraphNode : reference : 0


In [57]:
print("Transform points and display")
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
o3d.visualization.draw_geometries(pcds_down,
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])



Transform points and display
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 0.73185703 -0.3296033   0.59644526  0.48231718]
 [ 0.35164602  0.93237802  0.08376337  0.01587609]
 [-0.58372114  0.14843479  0.7982711  -0.16464156]
 [ 0.          0.          0.          1.        ]]


In [58]:
pcds = load_point_clouds(voxel_size)
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud("multiway_registration.pcd", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

